<a href="https://colab.research.google.com/github/TarunNathamBerkeley/StockMarketMLPredictor/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
from google.colab import drive
import os
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import datetime as dt
import re
import snscrape.modules.twitter as sntwitter
import yfinance as yf

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Data')
!ls

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Copy of data_processing.ipynb'
'Copy of Dow Jones Industrial Average Historical Data.csv'
'Copy of twitter_sentiment.csv'
 data_processing.ipynb
'Dow Jones Industrial Average Historical Data.csv'
'High Level Model.gdoc'
 model_training
 one-month-tweets.csv
 sentiment.csv
 tweets_labeled.csv
 tweets_remaining_09042020_16072020.csv
 twitter_sentiment.csv


In [ ]:
df = pd.read_csv('twitter_sentiment.csv', encoding_errors='replace')
df.columns=['label', 'tweet_id', 'date', 'query', 'handle', 'text']
df2 = pd.read_csv('Dow Jones Industrial Average Historical Data.csv')

In [ ]:
df2.Price = df2.Price.apply(lambda x: x.replace(',', ''))
df2.Price = pd.to_numeric(df2.Price)
df2.Date = pd.to_datetime(df2.Date)
df2.set_index('Date', inplace = True)


In [ ]:
#df2.loc['2019-11-01':'2019-12-31']
#df2[(df2['Date'] >= '2019-12-01') & (df2['Date'] <= '2019-12-31')]
#plt.plot(df2.Date, df2.Price)
#plt.show()

In [ ]:
X = df.text
y = df.label
#X = X.apply(word_tokenize)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_train.value_counts(), y_test.value_counts())

(1071999,) (528000,) (1071999,) (528000,)
0    536679
4    535320
Name: label, dtype: int64 4    264680
0    263320
Name: label, dtype: int64


In [ ]:
test_x = X.head(100)
test_x = test_x.apply(word_tokenize)

def removeStopWordsAndPunct(text):
  words_to_remove = set(stopwords.words('english')).union(set(string.punctuation))
  output = [word for word in text if word not in words_to_remove]
  return output

test_x = test_x.apply(removeStopWordsAndPunct)
test_x.head()

0    [upset, ca, n't, update, Facebook, texting, .....
1    [Kenichan, I, dived, many, times, ball, Manage...
2              [whole, body, feels, itchy, like, fire]
3    [nationwideclass, 's, behaving, 'm, mad, I, ca...
4                              [Kwesidei, whole, crew]
Name: text, dtype: object

In [ ]:
!pip install --upgrade git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-g1giq0_7
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-g1giq0_7
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 786815dd05681e2421cd03aa9acf5ab5c773bce9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#Get user input
def getTweets(start, end):
  query = '#TSLA'
  noOfTweet = '100'

  #db: [Day] [Vector of Tweets] [Stock mvmnt]

  tweets_list = []
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query + ' lang:en since:' +  start + ' until:' + end + ' -filter:links -filter:replies').get_items()):
                      if i > int(noOfTweet):
                          break
                      tweets_list.append(tweet.rawContent)

  return tweets_list


In [ ]:
# Download the data from Yahoo Finance API
import yfinance as yf

def getMvmnt(start, end):
  data = yf.download("TSLA", start=start, end=end)
  if data.empty:
    return None

  # Calculate the daily returns for TSLA
  daily_returns = data["Close"] - data["Open"]

  # Calculate the up/down movement for TSLA
  up_down = daily_returns.apply(lambda x: "Up" if x > 0 else "Down")

  return up_down.values[0]


In [ ]:
import datetime

start = datetime.datetime(2019, 1, 1) # replace with your start date and time
end = datetime.datetime(2023, 1, 1) # replace with your end date and time

dates = []

current = start
while current <= end:
    dates.append(current.strftime('%Y-%m-%d'))
    current += datetime.timedelta(days=1)

print(dates)


['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-26', '2019-01-27', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-02', '2019-02-03', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-09', '2019-02-10', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-23', '2019-02-24', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-02', '2019-03-03', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-09', '2019-03-10', '2019-03-11', '2019-03-12', '2019

In [ ]:
import yfinance as yf
data = yf.download("TSLA", start="2023-03-01", end="2023-03-02")
daily_returns = data["Close"] - data["Open"]
up_down = daily_returns.apply(lambda x: "Up" if x > 0 else "Down")
print(up_down.values[0])

[*********************100%***********************]  1 of 1 completed
Down


In [ ]:
data = []
for i in range(len(dates) - 1):
  start = dates[i]
  end = dates[i + 1]
  temp = []
  stockMovement = getMvmnt(start, end)
  if(stockMovement is not None):
    temp.append(start)
    temp.append(getTweets(start, end))
    temp.append(stockMovement)
    data.append(temp)
print(data)

In [ ]:
print(data)

[['2019-01-02', ['Grrr what does the market wants from #TSLA?  No other car company in the world is growing at the pace of #Tesla . It will continue 2 improve margins &amp; as battery costs decline further it will pass some savings 2 consumers -nothing wrong here', "Today on Mid Day Movers w/Lichtenstein today on the @TDANetwork we'll have a look at #GE #HOLX #FB #WYNN #LVS #XOM #CVX #CL_F #tsla and more - Check it out at 130pm ct - Be there or be... - Well - Just be there!", 'BLOCK TRADE detected in #TSLA', "I worked at $GE renewables in 2011, a period which saw revenue (primarily wind turbines) decline ~90% in 2 years due to cancellation of gov subsidies and rebates. To think that gov credit reductions won't impact #TSLA demand is utterly foolish. #TSLAQ", 'Stock drop: $TSLA and 0 other stocks drop. The Common Wealth is investigating! Reach out for more info. TheCommonWealth.mail@gmail.com #StockDrop #TSLA', 'What Time Today Does A #TESLA "Bullish" Email "Leak"? #ELON #MUSK #TSLA #DJ

In [ ]:

# Assume that 'data' is a list of lists with the structure:
# [Date, Tweets, Movement]

df = pd.DataFrame(data, columns=['Date', 'Tweets', 'Movement'])
for corpus in df['Tweets']:
  print(corpus)
df.shape

In [ ]:
df.drop("Tweets-String", axis=1)

In [ ]:
df.drop("Movement-Numerical", axis=1)

,Date,Tweets,Movement,Tweets-String
0,2019-01-02,[Grrr what does the market wants from #TSLA? ...,Up,Grrr what does the market wants from #TSLA? N...
1,2019-01-03,[Bought to open $TSLA at 299.00 and added to m...,Down,Bought to open $TSLA at 299.00 and added to my...
2,2019-01-04,[],Up,
3,2019-01-07,"[Rally holding on in the final hour, #Dow +118...",Up,"Rally holding on in the final hour, #Dow +118 ..."
4,2019-01-08,[Q for #TSLA tweeps: Do you plug-in for &lt;1 ...,Down,Q for #TSLA tweeps: Do you plug-in for &lt;1 h...
...,...,...,...,...
596,2021-05-14,[Wonder if Air Pollution has anything to do wi...,Up,Wonder if Air Pollution has anything to do wit...
597,2021-05-17,"[Buy #dogecoin\nBuy #tsla, #ElonMusk's #Tesla ...",Up,Buy #dogecoin\nBuy #tsla #ElonMusk's #Tesla #T...
598,2021-05-18,"[Bought #tsla *and* #doge today, Hey @elonmusk...",Up,Bought #tsla *and* #doge today Hey @elonmusk j...
599,2021-05-19,[Within a few weeks\nBest upcoming trade #tsla...,Up,Within a few weeks\nBest upcoming trade #tsla ...


In [ ]:
df = df.drop("Tweets-String", axis=1)

In [ ]:
drive.mount('/drive')

In [ ]:
df.to_csv('/drive/My Drive/Data/tweet_data_two_years.csv')

In [ ]:
df.head()

,Date,Tweets,Movement
0,2019-01-02,[Grrr what does the market wants from #TSLA? ...,Up
1,2019-01-03,[Bought to open $TSLA at 299.00 and added to m...,Down
2,2019-01-04,[],Up
3,2019-01-07,"[Rally holding on in the final hour, #Dow +118...",Up
4,2019-01-08,[Q for #TSLA tweeps: Do you plug-in for &lt;1 ...,Down


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Create a list to store the vector for each tweet
tweet_vectors = []

df['Tweets-String'] = df['Tweets'].apply(lambda x: " ".join(x))

# corpus = df['Tweets-String']
# x = vectorizer.fit_transform(corpus)
# print(x)

# df['Tfidf'] = x

In [ ]:
df.head()

,Date,Tweets,Movement,Tweets-String
0,2019-01-02,[Grrr what does the market wants from #TSLA? ...,Up,Grrr what does the market wants from #TSLA? N...
1,2019-01-03,[Bought to open $TSLA at 299.00 and added to m...,Down,Bought to open $TSLA at 299.00 and added to my...
2,2019-01-04,[],Up,
3,2019-01-07,"[Rally holding on in the final hour, #Dow +118...",Up,"Rally holding on in the final hour, #Dow +118 ..."
4,2019-01-08,[Q for #TSLA tweeps: Do you plug-in for &lt;1 ...,Down,Q for #TSLA tweeps: Do you plug-in for &lt;1 h...


In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

stringToNum = {"Down": 0, "Up": 1}
df['Movement-Numerical'] = df['Movement'].apply(lambda x: stringToNum[x])


pipeline = Pipeline([
  ("vectorizer", TfidfVectorizer()),
  ("classifier", XGBClassifier())
])

X = df['Tweets-String'].values
y = df['Movement-Numerical'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False, stratify = None)

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
accuracy = accuracy_score(preds, y_test)
accuracy

0.5785123966942148

In [ ]:
df.count()

Date                  601
Tweets                601
Movement              601
Movement-Numerical    601
Tweets-String         601
dtype: int64

In [ ]:
from google.colab import  drive

In [ ]:
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
df.to_csv('/drive/My Drive/Data/tweet_data_two_years.csv')

# DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

stringToNum = {"Down": 0, "Up": 1}
df['Movement-Numerical'] = df['Movement'].apply(lambda x: stringToNum[x])


pipeline = Pipeline([
  ("vectorizer", TfidfVectorizer()),
  ("classifier", DecisionTreeClassifier())
])

X = df['Tweets-String'].values
y = df['Movement-Numerical'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False, stratify = None)

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
accuracy = accuracy_score(preds, y_test)
accuracy

0.5371900826446281

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in 

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, ClassLabel

# Helper method to tokenize text.
def tokenize_function(examples):
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Helper method to tokenize given dataset.
def tokenize_data(data):
    dataset = Dataset.from_dict({"label":data['label'] , "text": data['text'].values})
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.cast_column("label", ClassLabel(names = ["0","1"]))
    return tokenized_dataset

# Helper method for Trainer.
def compute_metrics(p):
    logits, labels = p
    pred = np.argmax(logits, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy}

  # Helper method to initiate a new Trainer with given train and test sets.
def get_trainer(train_set, test_set):

    # Model params.
    model_name = "distilbert-base-uncased"
    model_folder = "model_training"
    max_training_steps = 300
    num_classes = 2

    # Set training args.
    # We time-seed to ensure randomness between different benchmarking runs.
    training_args = TrainingArguments(
        max_steps=max_training_steps,
        output_dir=model_folder,
        seed = int(datetime.datetime.now().timestamp())
    )

    # Tokenize train/test set.
    train_tokenized_dataset = tokenize_data(train_set)
    test_tokenized_dataset = tokenize_data(test_set)

    # Initiate a pre-trained model.
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics = compute_metrics,
        train_dataset = train_tokenized_dataset,
        eval_dataset = test_tokenized_dataset,
    )
    return trainer

transformer_df = df[['Tweets-String', 'Movement-Numerical']]
transformer_df.columns = ['text', 'label']
train_set, test_set = train_test_split(transformer_df, test_size=0.2, random_state=42, shuffle = False, stratify = None)
trainer = get_trainer(train_set, test_set)
trainer.train()
eval_metrics = trainer.evaluate()
acc = eval_metrics['eval_accuracy']

In [ ]:
acc

In [ ]:
df.to_csv("tweets_v1.csv", index=False)

OSError: ignored

In [ ]:
import tensorflow as tf

# Get the tweet vectors from the DataFrame
X = np.array(df['TweetVec'].tolist())
y = np.array(df['Movement'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)
